In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-6e247b23-2bb3-e804-d578-5c965005fd37)


#install pacakges

In [2]:
%%capture
!pip install ktrain


# download data

In [3]:
import os
if os.path.isdir('/content/Profiling-Hate-Speech-Spreaders-on-Twitter') is False:
    !git clone https://github.com/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter.git

In [4]:
with open('pass.txt','r') as f:
  pas=f.read()

In [5]:
import zipfile
if os.path.isdir('train') is False:
  train_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/train.zip'
  with zipfile.ZipFile(train_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))
if os.path.isdir('test') is False:
  test_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/test.zip'
  with zipfile.ZipFile(test_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))

# prepare data

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
df=pd.read_csv('train/es/truth.txt',sep=':::',header=None,engine='python')
df.columns=['id','label']

In [7]:
df.label.value_counts()

1    100
0    100
Name: label, dtype: int64

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preprocessing(text):
    text=text.str.replace('\d+', '')
    text=text.str.replace('RT','')
    text=text.str.replace('#USER#','')
    text=text.str.replace('#URL#','')
    text= text.str.lower()
    text = text.str.replace('[^\w\s]','')
    #text = text.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
    #text = text.apply(lambda x: [item for item in x if item not in stop])
    #text = text.apply(lambda x : " ".join(x))
    return text

In [10]:
df.id='train/es/'+df.id.astype(str)+'.xml'
df.head()

,id,label
0,train/es/1a698d93874644625d83a76b206c7aa3.xml,0
1,train/es/b7200193802bde3769f4daa371b5305b.xml,0
2,train/es/7ffb758291340303c2162fcee25e2780.xml,0
3,train/es/e309e9e2516824370310661c919dd5d8.xml,0
4,train/es/b523abc590889dfaf69a1c8470ef314d.xml,0


In [11]:
len(df)

200

In [12]:
import xml.etree.ElementTree as ET
def reader(df,ground=True):
  data=[]
  for x in df.iterrows():
      
      tree = ET.parse(x[1].id)
      root = tree.getroot()
      text=[x.text for x in root[0]]
      if ground:
        label=[x[1].label]*len(text)
        data.append(pd.DataFrame(zip(text,label),columns=['text','label']))
      else:
        data.append(pd.DataFrame(text,columns=['text']))
  return data

In [13]:
data=reader(df)

In [14]:
data[0].head()

,text,label
0,RT #USER#: Esperando para ver la captura de Ja...,0
1,#USER# #USER# Que bueno Romeo con la ayuda De ...,0
2,RT #USER#: Nuestras cuadrillas están restauran...,0
3,RT #USER#: Esta es la portada de #HASHTAG# de ...,0
4,RT #USER#: Lo más matado de la risa que vas a ...,0


In [15]:
len(data),len(data[0])

(200, 200)

In [16]:
from sklearn.model_selection import train_test_split
trainx,valx=train_test_split(data,test_size=0.1,)
trainx,testx=train_test_split(trainx,test_size=0.1)

In [17]:
train=pd.concat(trainx)
val=pd.concat(valx)
test=pd.concat(testx)

In [18]:
train.label.value_counts()

1    16400
0    16000
Name: label, dtype: int64

In [19]:
val.label.value_counts()

1    2000
0    2000
Name: label, dtype: int64

In [20]:
test.label.value_counts()

0    2000
1    1600
Name: label, dtype: int64

In [21]:
train.text=preprocessing(train.text)
val.text=preprocessing(val.text)
test.text=preprocessing(test.text)

In [22]:
len(train),len(val),len(test)

(32400, 4000, 3600)

In [23]:
train = train.sample(frac=1).reset_index(drop=True)
val = val.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)


In [24]:
train_text=train.text.values
train_label=train.label.values
val_text=val.text.values
val_label=val.label.values
test_text=test.text.values
test_label=test.label.values

# classify data

In [25]:
from sklearn.model_selection import StratifiedKFold
from ktrain import text
import ktrain
MODEL_NAME = 'dccuchile/bert-base-spanish-wwm-uncased'
t = text.Transformer(MODEL_NAME, maxlen=20,classes=[1,0])
trn = t.preprocess_train(train_text, train_label)
tst = t.preprocess_test(val_text, val_label)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=tst, batch_size=16)
learner.fit_onecycle(0.00001, 5) 
print(learner.validate(class_names=t.get_classes()))

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:414: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')



preprocessing train...
language: es
train sequence lengths:
	mean : 11
	95percentile : 21
	99percentile : 24


Is Multi-Label? False
preprocessing test...
language: es
test sequence lengths:
	mean : 11
	95percentile : 21
	99percentile : 25





begin training using onecycle policy with max lr of 1e-05...
Epoch 1/5
2025/2025 [==============================] - 451s 210ms/step - loss: 0.6335 - accuracy: 0.6325 - val_loss: 0.6367 - val_accuracy: 0.6535
Epoch 2/5
2025/2025 [==============================] - 433s 210ms/step - loss: 0.5739 - accuracy: 0.6994 - val_loss: 0.6407 - val_accuracy: 0.6593
Epoch 3/5
2025/2025 [==============================] - 433s 209ms/step - loss: 0.4963 - accuracy: 0.7591 - val_loss: 0.7125 - val_accuracy: 0.6528
Epoch 4/5
2025/2025 [==============================] - 433s 210ms/step - loss: 0.3422 - accuracy: 0.8541 - val_loss: 0.8941 - val_accuracy: 0.6442
Epoch 5/5
2025/2025 [==============================] - 431s 209ms/step - loss: 0.2111 - accuracy: 0.9171 - val_loss: 1.1191 - val_accuracy: 0.6438
              precision    recall  f1-score   support

           1       0.64      0.65      0.65      2000
           0       0.65      0.64      0.64      2000

    accuracy                         

In [26]:
val = t.preprocess_test(val_text, val_label)

val_pred = learner.predict(val)
print(classification_report(val_label,np.argmax(val_pred,axis=1)))

preprocessing test...
language: es
test sequence lengths:
	mean : 11
	95percentile : 21
	99percentile : 25


              precision    recall  f1-score   support

           0       0.64      0.65      0.65      2000
           1       0.65      0.64      0.64      2000

    accuracy                           0.64      4000
   macro avg       0.64      0.64      0.64      4000
weighted avg       0.64      0.64      0.64      4000



In [27]:
test = t.preprocess_test(test_text, test_label)

val_pred = learner.predict(test)
print(classification_report(test_label,np.argmax(val_pred,axis=1)))

preprocessing test...
language: es
test sequence lengths:
	mean : 11
	95percentile : 20
	99percentile : 24


              precision    recall  f1-score   support

           0       0.62      0.60      0.61      2000
           1       0.51      0.53      0.52      1600

    accuracy                           0.57      3600
   macro avg       0.56      0.57      0.56      3600
weighted avg       0.57      0.57      0.57      3600



# validation

In [ ]:
preds,trues=[],[]
for i in valx:
  pred=list(clf.predict(i.text.values,verbose=0))
  preds.append(max(pred,key=pred.count))
  trues.append(int(np.mean(i.label)))
print(classification_report(trues,preds))  

In [ ]:
preds,trues=[],[]
for i in testx:
  pred=list(clf.predict(i.text.values,verbose=0))
  preds.append(max(pred,key=pred.count))
  trues.append(int(np.mean(i.label)))
print(classification_report(trues,preds))  

# test evaluation

In [ ]:
from glob import glob
test_path=pd.DataFrame(glob('test/en/*.xml'),columns=['id'])
test_path.head()

,id
0,test/en/f2973063f16c0698a6de86c55b5f9ad6.xml
1,test/en/be71e41921653e523aceb555727b61c7.xml
2,test/en/d0235c3e74f79298fc2b6d2427c2060b.xml
3,test/en/314d4bf068314367bc83dc5ae3757e13.xml
4,test/en/fdd27881da241cff7115c1e89caba6dd.xml


In [ ]:
test=reader(test_path,ground=False)

In [ ]:
test[0].head()

,text
0,#USER# I'm so sorry 😞...
1,#USER# #USER# That is correct.
2,RT #USER#: The legitimate President of the Uni...
3,#USER# #USER# #USER# Texas AG is suing the Bid...
4,#USER# In Jesus Holy Name. Soon the entire wor...


In [ ]:
final_preds=[]
for i in range(len(test)):
  test[i].text=preprocessing(test[i].text)
  pred=list(clf.predict(test[i].text.values,verbose=0))
  final_preds.append(int(max(pred,key=pred.count)))


In [ ]:
len(final_preds),len(test_path)

In [ ]:
ids=test_path.id.apply(lambda x:x.split('/')[-1].split('.')[0])

In [ ]:
lang=['en']*len(test_path)

In [ ]:
final=pd.DataFrame(zip(ids,lang,final_preds),columns=['author-id','lang','type'])
final.head()